In [25]:
import torch
import torchvision
import torchvision.transforms as transforms

In [26]:
transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=True,
                                        download=True, transform=transform)
#trainset.data = trainset.data[:]
#trainset.targets = trainset.targets[:]
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [27]:
from rac.utils.models import cifar10net, resnet

In [28]:
import torch.nn as nn
import torch.nn.functional as F

In [20]:
model1 = resnet.ResNet18()
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    outputs, e = model1(inputs, last=True)
    print(outputs.shape, e.shape)
    break

torch.Size([128, 10]) torch.Size([128, 512])


In [11]:
model1 = cifar10net.CifarNet()
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    outputs, e = model1(inputs, last=True)
    print(outputs.shape, e.shape)
    break

torch.Size([256, 10]) torch.Size([256, 256])


In [29]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [30]:
device

device(type='cuda', index=0)

In [31]:
import torch.optim as optim
#model1 = cifar10net.CifarNet()
model1 = resnet.ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.001, momentum=0.9)

In [34]:
for epoch in range(100):  # loop over the dataset multiple times
    print(epoch)

    running_loss = 0.0
    step = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device),

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        step += 1
    print("loss: ", running_loss/step)
    step = 0
    running_loss = 0.0
print('Finished Training')

0
loss:  0.7348631565528148
1
loss:  0.6614138560984141
2
loss:  0.6002557455273845
3
loss:  0.5546936908798754
4
loss:  0.5155701359824452
5
loss:  0.47662327341411426
6
loss:  0.4573782105427569
7
loss:  0.42360721806735946
8
loss:  0.40217307667293206
9
loss:  0.38211964772028084
10
loss:  0.3593208087069909
11
loss:  0.3418885225149067
12
loss:  0.32182869761038924
13
loss:  0.30784662300363524
14
loss:  0.29128827089848724
15
loss:  0.27800126131766895
16
loss:  0.2666052648478457
17
loss:  0.2545077907269263
18
loss:  0.24272228272445975
19
loss:  0.2323410873446623
20
loss:  0.22880115255218028
21
loss:  0.21130731012052892
22
loss:  0.2014266756527564
23
loss:  0.19171059708994673
24
loss:  0.18309223680468775
25
loss:  0.17345810243312051
26
loss:  0.16821913497374796
27
loss:  0.1575790066891314
28
loss:  0.15677520485066088
29
loss:  0.14264267703989889
30
loss:  0.1414374805548612
31
loss:  0.13334055825153276
32
loss:  0.1271929840040405
33
loss:  0.12054512515435438
34
lo

In [36]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = model1(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 89 %
